## Code Summary

The following code reduces the dimensionality of structural and compositional neighborhood graphs and uses agglomerative hierarchical clustering to partition the low-dimensional spaces and assign classifications to the resulting partitions

Specifically, this code:

**(1)** Takes as input ".gdv" files and "vapor.npy" files that (i) contain structural and compositional neighborhood graphs and (ii) identify vapor particles. These files can describe multiple trajectories with particles that have different interaction potentials, sizes, etc. Note that the corresponding ".xyz" files should be in the same folder.

**(2)** Trains a deep neural network called an "autoencoder" (using only the unique neighborhood graphs). An "encoder" is extracted from the "autoencoder" and is then used for dimensionality reduction. Note that both "structural" and "compositional" autoencoders/encoders are created.

**(3)** Uses agglomerative hierarchical clustering (via Ward's linkage) to partition the structural and compositional low-dimensional spaces and assign classifications to the resulting partitions.

**(4)** Modifies the ".xyz" files such that individual particles are colored according to their discrete classifications. These ".xyz" files can be visualized in OVITO. Qualitative analyses of these images can provide a general idea of what each class physically represents.

**Note**: Example data and results of this entire framework can be found in the "Example" folder. This folder contains a "filled out" example of this code (called Train_example.ipynb) that we recommend reading through. 

In [ ]:
# Do not write bytecode to maintain clean directories
import sys
sys.dont_write_bytecode = True

# Import required packages and helper functions
import os
import core
import importlib
# importlib.reload(core)

# Prepare TensorFlow
core.prepare_tensorflow()

# import importlib
# importlib.reload(core)

## Choose name of folder where all classification results will be stored

This folder will be referred to as the "mother directory" going forward

In [ ]:
# Create mother directory
mother_dir = "Your Directory Name"
os.mkdir(mother_dir)

## Place .gdv, .xyz, and vapor.npy files for all trajectories in unique folders

An ".xyz" file for each given trajectory should be processed according to the crayon package discussed/shown in the "BGD_Example" folder. These ".xyz" files and corresponding ".gdv" and "vapor.npy" files should all be placed in the same folder. For example, an ".xyz" file named "cry_x_ds.xyz" should be placed in a folder labeled "cry_x_ds" along with its ".gdv" and "vapor.npy" files. Please make sure that the .xyz files are the "extended" .xyz format and at a minimum contain data for species type, radius, and position coordinates. See "Example_Data.zip" within the "Example" folder for an example. 

Record the names of these folders once they are created.

In [ ]:
# Create list of directories with ".gdv", ".xyz", and "vapor.npy" files for each trajectory of interest
traj_dirs = []
traj_dirs.append("ss/Cry_x_ss")
traj_dirs.append("ss/Cry_x_ss") # etc. Add as many trajectories as you would like

## Extract unique structural and compositional neighborhood graphs.

Note that neighborhood graphs of vapor particles are not recorded.

In [ ]:
# Note that "st" refers to "structural" and "co" refers to "compositional
[gdv_unique_st, 
 gdv_unique_co]= core.process_gdvs_train(traj_dirs,
                                         mother_dir)

## Split unique neighborhood graphs into training/validation/testing data sets

This function first normalizes the unique neighborhood graphs (by weighing them to account for correlations between individual graphlet nodes) and then scales these weighted neighborhood graphs from -1 to 1. The unique, normalized neighborhood graphs are split into training/validation/testing data sets. that are used to train/validate/test the structural and compositional autoencoders. Note that the validation and testing data set sizes are constrained to be identical. Here, a 60%/20%/20% split is chosen as an example.

In [ ]:
train_per = 60 # % of data points used in training data
[x_train_st, 
 x_val_st, 
 x_test_st, 
 min_st, 
 max_st] = core.train_prep(gdv_unique_st, 
                           train_per, 
                           mother_dir,
                           model_type="struct")

[x_train_co, 
 x_val_co, 
 x_test_co, 
 min_co, 
 max_co] = core.train_prep(gdv_unique_co, 
                           train_per, 
                           mother_dir,
                           model_type="comp")

## Explore architectural choices for structural and compositional autoencoders.

Enter potential autoencoder architectural choices as a list. The code will train separate autoencoders for each of these combinations and output an "elbow" plot that compares their performance. Each model will be saved along with the training loss and validation loss at each epoch, testing loss, and total training time in the mother directory. Other architectural choices (e.g., dropout probability, learning rate) can be adjusted in core.py. 

In [ ]:
# Make architectural choices
# The architecture choices below represent our suggestions -- but
# feel free to change these
h_nodes = [25, 50, 100, 200] # number of nodes per hidden layer
bn_nodes = [1, 2, 3, 4, 5, 6] # number of bottleneck layer nodes
h_layers = [2] # number of hidden layers
patience_st = 50 # patience for early stopping. We suggeset between 25-250,
# with smaller patiences being used for larger data sets
patience_co = 100

# Train structural and compositional autoencoders
core.train_autoencoders(x_train_st, 
                        x_val_st, 
                        x_test_st, 
                        h_nodes, 
                        bn_nodes, 
                        h_layers, 
                        patience_st, 
                        mother_dir)

core.train_autoencoders(x_train_co, 
                        x_val_co, 
                        x_test_co, 
                        h_nodes, 
                        bn_nodes, 
                        h_layers, 
                        patience_co, 
                        mother_dir)
   

# Create elbow plots -- only options for model type are "struct" and "comp"
core.create_elbow(mother_dir, 
                  model_type = "struct")

core.create_elbow(mother_dir, 
                  model_type = "comp")

## Make final autoencoder architectural choices

After examining the elbow plots, choose the "final" architectural choices for both the structural and compositional autoencoders below. Note that the "encoders" are extracted from the "autoencoders" for dimensionality reduction.

In [ ]:
# Make final architectural choices for structrual autoencoder
hn_st = 'Write Integer Here' # number of nodes per hidden layer
bn_st = 'Write Integer Here' # number of bottleneck layer nodes
hl_st = 'Write Integer Here' # number of hidden layers

# Make final architectural choices for compositional autoencoder
hn_co = 'Write Integer Here' # number of nodes per hidden layer
bn_co = 'Write Integer Here' # number of bottleneck layer nodes
hl_co = 'Write Integer Here' # number of hidden layers

# Load chosen structural and compositional encoders (and directories
# in which these encoders are saved)
[enc_st, 
 enc_dir_st] = core.get_encoder(hn_st, 
                                bn_st, 
                                hl_st, 
                                mother_dir, 
                                model_type = "struct")
[enc_co, 
 enc_dir_co] = core.get_encoder(hn_co, 
                                bn_co, 
                                hl_co, 
                                mother_dir, 
                                model_type = "comp")

## Reduce dimensionality of the neighborhood graphs using the  models

Now that the encoder models are chosen, the dimensionality of the unique neighborhood graphs and the neighborhood graphs corresponding to each particle in each .xyz/.gdv file is reduced. 

**Note** All results are saved under the folder of the appropriate autoencoder architecture (i.e., mother_dir/Autoencoder_Struct/xx_HL_xx_Nodes/xx_OP or mother_dir/Autoencoder_Comp/xx_HL_xx_Nodes/xx_OP). This trend continues for the rest of the code blocks.

In [ ]:
# Reduce dimensionality of unique neighborhood graphs
lowd_st_unique = core.reduce_dim_uniquegdvs(enc_st, 
                                            gdv_unique_st, 
                                            min_st, 
                                            max_st, 
                                            enc_dir_st)

lowd_co_unique = core.reduce_dim_uniquegdvs(enc_co, 
                                            gdv_unique_co, 
                                            min_co, 
                                            max_co, 
                                            enc_dir_co)

# Reduce dimensionality of each neighborhood graph of each particle in each .xyz file
lowd_all_st = {}
for traj_dir in traj_dirs:
    lowd_all_st[traj_dir] = core.reduce_dim_allgdvs(enc_st, 
                                                    min_st, 
                                                    max_st, 
                                                    traj_dir, 
                                                    enc_dir_st)

lowd_all_co = {}
for traj_dir in traj_dirs:
    lowd_all_co[traj_dir] = core.reduce_dim_allgdvs(enc_co, 
                                                    min_co, 
                                                    max_co,
                                                    traj_dir, 
                                                    enc_dir_co)

## Create cluster trees
Agglomerative hierarchical clustering (via Ward's linkage) is next implemented to cluster the low-dimensional representations of all unique neighborhood graphs. The first step here is to create a cluster tree, which is done below.

In [ ]:
# Calculate cluster trees (i.e., linkage) based on low-dimensional representations of
# unique stuctural and compositional neighborhood graphs
Z_st = core.calc_linkage(lowd_st_unique)
Z_co = core.calc_linkage(lowd_co_unique)

## Choose the "best" number of clusters for classification
Because agglomerative hierarchical clustering creates a "cluster tree", it is important to find the "best" number of clusters for classification.

To choose the "best" number of clusters, the code below plots the number of low-dimensional points (i.e., neighborhood graphs) corresponding to "target lattices" against the number of clusters in each branch of the resulting cluster tree. The "best' cluster number can be chosen as the point in which the target lattice cluster sizes (qualitatively) stabilize. 

Here, a "target lattice" cluster is defined as a cluster that contains a low-dimensional coordinate that corresponds to a theoretically perfect lattice. So, a "BCC" cluster contains the low-dimensional representation of the theoretically perfect BCC neighborhood graph.

The attached works classifiy colloidal lattices that form FCC, HCP, BCC, DCsCl, and IrV-like structures. In this code, the target choices currently available are:

Structural: "FCC", "HCP", "BCC", "IrVA", "IrVB", "DCsClA", "DCsClB", "Weak"
Compositional: "FCC_HCP_IrVB", "BCC_DCsClB", "IrVA_DCsClA"

However, more structures can easily be added into core.py.

**Note #1**: We recognize that the user's trajectory data may form BCC/FCC/HCP-like structures but not actually contain particles that have "theoretically perfect" lattice neighborhood graphs/low-dimensional coordinates. As a result, the target lattice cluster is defined as a cluster that contains the low-dimensional coordinate with the smallest distance to its theoretically perfect analog. For example, let's say the low-dimensional representation of the perfect BCC lattice is [1, 1, 1]<sup>T</sup>, but only [1 ,1, 1.01]<sup>T</sup> exists in the provided data. Then, [1, 1, 1.01]<sup>T</sup> will be treated as the "perfect" BCC coordinate.

**Note #2**: We further recognize that the user may not want to choose the cluster number based on the use of target lattices. If this is the case, feel free to skip this step.

**Note #3**: Notice that certain structures (e.g., FCC/HCP/IrVB) have identical compositional neighborhood graphs.


In [ ]:
# Choose target lattices for structrual and compositional models
target_st = ["FCC", "HCP", "BCC"]
target_co = ["FCC_HCP_IrVB", "BCC_DCsClB"]

In [ ]:
# Plot populations of structural and compositional target clusters versus
# the total number of clusters in each branch of the cluster tree
# Note that the indices of the these target clusters are also returned.
# These indices are useful for tracking/book-keeping
[lowd_target_st, 
 target_ind_st, 
 clust_count_st] = core.choose_cluster_num(enc_st,
                                           target_st, 
                                           min_st, 
                                           max_st, 
                                           lowd_st_unique, 
                                           Z_st,
                                           enc_dir_st, 
                                           model_type="struct")


[lowd_target_co, 
 target_ind_co, 
 clust_count_co] = core.choose_cluster_num(enc_co,
                                           target_co, 
                                           min_co, 
                                           max_co, 
                                           lowd_co_unique, 
                                           Z_co,
                                           enc_dir_co, 
                                           model_type="comp")

## Cluster the low-dimensional spaces

Based on the above plots (or not), choose the number of structural and compositonal clusters you would like to continue with. The code below then clusters these low-dimensional spaces according to this cluster number. The code even outputs associated dendograms that show the hierarchical structure of the resulting clusters.

In [ ]:
# Choose number of clusters for structural and compositional spaces
nc_st = 'Write Integer Here'
nc_co = 'Write Integer Here'

# Cluster structural and compositional low-dimensional spaces. Note that these
# are the cluster ids of the unique neighborhood graphs
clust_ids_st = core.cluster(lowd_st_unique, 
                            Z_st, 
                            target_ind_st, 
                            nc_st, 
                            enc_dir_st, 
                            model_type="struct")

clust_ids_co = core.cluster(lowd_co_unique, 
                            Z_co, target_ind_co, 
                            nc_co, 
                            enc_dir_co, 
                            model_type="comp")

In [ ]:
# Assign structrual and compositional clusters to all particles in all .xyz files
clust_ids_all_st = {}
clust_ids_all_co = {}
for traj_dir in traj_dirs:
    clust_ids_all_st[traj_dir] = core.assign_clusters(lowd_st_unique, 
                                                     clust_ids_st, 
                                                     lowd_all_st[traj_dir],
                                                     traj_dir, 
                                                     enc_dir_st)
    
    clust_ids_all_co[traj_dir] = core.assign_clusters(lowd_co_unique,
                                                     clust_ids_co, 
                                                     lowd_all_co[traj_dir],
                                                     traj_dir, 
                                                     enc_dir_co)

In [ ]:
# Get cluster labels of target lattices for structrual and compositional models
target_clust_st = core.get_target_clusters(clust_ids_st, 
                                           target_ind_st)

target_clust_co = core.get_target_clusters(clust_ids_co, 
                                           target_ind_co)

## Combined Classification

Now the structural and compositional low-dimensional spaces have been clustered, it is time to classify the particles in the XYZ files. We determine this classification based on the chosen target lattices, and structural and compositional clusters.For example, if a given particle’s structural and compositional low-dimensional representations fall under “FCC” identified clusters, the particle will be labeled as a “structurally and compositionally ordered FCC particle” (i.e., “CO-FCC”). If only the particle’s structural low-dimensional representation falls under an FCC cluster, the particle will be labeled as “structurally ordered, yet compositionally disordered FCC” (i.e., “CD-FCC”). If the particle’s low dimensional representation does not fall under any target lattice cluster, the particle is left unlabeled. These “unlabeled” particles often correspond to vapor particles, structurally defective particles, surface particles, etc. Note that we will use the "CO" and "CD" abbreviations in the next few code blocks.

In [ ]:
# Based on target lattices from before, we create and assign clusters for "CO" and "CD"
# target lattices.
target_comb = core.get_combined_cluster_ids(target_clust_st, target_clust_co)

In [ ]:
#Classify all particles in all provided trajectories
clust_ids_all_comb = {}
for traj_dir in traj_dirs:
    clust_ids_all_comb[traj_dir] = core.assign_clusters_combined(target_comb,
                                                                clust_ids_all_st[traj_dir], 
                                                                clust_ids_all_co[traj_dir], 
                                                                traj_dir, 
                                                                enc_dir_st)

# OVITO Visualization and Classification

The original .xyz files are re-written such that each particle is colored according to its classification. These .xyz files can be visualized in OVITO. Use the qualitative analyses from these visualizations along with the cluster tree to assign physical meaning(s) to each class.

**Note** The re-written  are saved under the folder of the appropriate structural autoencoder architecture (i.e., mother_dir/Autoencoder_Struct/xx_HL_xx_Nodes/xx_OP/traj_dir)


In [ ]:
# First assign colors to combined cluster IDs. Note that this function assigns "true"
# colors to "CO" particles, light colors to "CD" particles, and white to unabeled particles.
# Feel free to modify this in the core.py file. For example, CO-FCC could be red, while CD-
# CD-FCC would be light red. The function outputs a list of used colors (and their RGB
# values) and their corresponding classifications.
colors_list, colors_dict = core.assign_colors(target_comb,
                                              mother_dir)

# Print color/classification assignments for user clarity. Dictionary is also saved as
# text file
print (colors_dict)

In [ ]:
# Write XYZ files 
for traj_dir in traj_dirs:
    core.create_XYZ(clust_ids_all_comb[traj_dir], 
                    colors_list, 
                    traj_dir, 
                    enc_dir_st)